In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
train = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")
test = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
sample_submission = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv")

In [44]:
train.head()

In [45]:
test.head()

In [46]:
sample_submission.head()

In [47]:
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
A = train.groupby(by = ['language']).count()
A

In [49]:
# define Seaborn color palette to use
colors = sns.color_palette('pastel')[0 : 5]

# create text props
textprops = dict(
    horizontalalignment = "center",
    verticalalignment = "top",
    rotation = 0,
    rotation_mode = "anchor",
    # rotation_mode = "default", 
    size = 14,
    color = "#81D8D0"
)
# create pie chart
plt.figure(figsize=(6, 6))
plt.pie(x = A.id,
        labels = A.index,
        colors = colors,
        autopct='%.2f%%',
        explode = [0.02] * 2,
        startangle = 90,
        pctdistance = 0.4,
        labeldistance = 1.2,
        textprops = textprops, 
       )

# draw circle
centre_circle = plt.Circle((0,0),0.70,fc = 'white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()